In [231]:
import pyarrow as pa
from pyarrow import csv, parquet
from pyarrow import dataset as pads
from pathlib import Path
import pandas as pd

from config import PolygonConfig

In [221]:
environ = {
    # "POLYGON_DATA_DIR": "../data/polygon",
    "POLYGON_DATA_DIR": "/Volumes/Oahu/Mirror/files.polygon.io",
    "POLYGON_AGG_TIME": "day",
}

config = PolygonConfig(
    environ=environ,
    calendar_name="XNYS",
    start_session="2010-10-07",
    end_session="2010-10-15",
    # start_session="2003-10-01",
    # start_session="2018-01-01",
    # start_session="2023-01-01",
    # end_session="2023-01-12",
    # end_session="2023-12-31",
    # end_session="2024-06-30",
)

In [222]:
test_csv_path = Path(config.aggs_dir) / "2020/10/2020-10-07.csv.gz"
print(f"{test_csv_path.exists()=}")
test_csv_path

test_csv_path.exists()=True


PosixPath('/Volumes/Oahu/Mirror/files.polygon.io/flatfiles/us_stocks_sip/day_aggs_v1/2020/10/2020-10-07.csv.gz')

The Polygon historical flatfile docs say Minute Aggregates have millisecond timestamps (like the Day Aggregates) but they actually have nanosecond timestamps (like Trades).
https://polygon.io/flat-files/stocks-min-aggs?stocks-min-aggs=documentation

I reported this to them in June 2024 and it (the docs, I don't think the resolution should change) hasn't been corrected as of September (I did ping them again).

Actually just discovered the Day Aggs are nanosecond resolution also (I had been working only on minute bars).

In [223]:
# Polygon Aggregate flatfile timestamps are in nanoseconds (like trades), not milliseconds as the docs say.
# I make the timestamp timezone-aware because that's how Unix timestamps work and it may help avoid mistakes.
timestamp_type = pa.timestamp('ns', tz='UTC')

# But we can;t use the timestamp type in the schema here because it's not supported by the CSV reader.
# So we'll use int64 and cast it later.

# strptime(3) (used by CSV reader for timestamps in ConvertOptions.timestamp_parsers) supports Unix timestamps (%s) and milliseconds (%f) but not nanoseconds.
# https://www.geeksforgeeks.org/how-to-use-strptime-with-milliseconds-in-python/
# Actually that's the wrong strptime (it's Python's).  C++ strptime(3) doesn't even support %f.
# https://github.com/apache/arrow/issues/39839#issuecomment-1915981816
# Also I don't think you can use those in a format string without a separator.
# This seems weird given (Py)Arrow's high-performance focus so maybe I'm missing something.

# Polygon price scale is 4 decimal places (i.e. hundredths of a penny), but we'll use 10 because we have precision to spare.
# price_type = pa.decimal128(precision=38, scale=10)
price_type = pa.float64()

polygon_aggs_schema = pa.schema([
    pa.field('ticker', pa.string(), nullable=False),
    pa.field('volume', pa.int64(), nullable=False),
    pa.field('open', price_type, nullable=False),
    pa.field('close', price_type, nullable=False),
    pa.field('high', price_type, nullable=False),
    pa.field('low', price_type, nullable=False),
    pa.field('window_start', pa.int64(), nullable=False),
    pa.field('transactions', pa.int64(), nullable=False),
])
polygon_aggs_schema

ticker: string not null
volume: int64 not null
open: double not null
close: double not null
high: double not null
low: double not null
window_start: int64 not null
transactions: int64 not null

In [224]:
convert_options = csv.ConvertOptions(
    column_types=polygon_aggs_schema,
    strings_can_be_null=False,
    quoted_strings_can_be_null=False,
)
table = csv.read_csv(test_csv_path, convert_options=convert_options)

table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: int64
transactions: int64
----
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[1602043200000000000,1602043200000000000,1602043200000000000,1602043200000000000,1602043200000000000,...,1602043200000000000,1602043200000000000,1602043200000000000,1602043200000000000,1602043200000000000]]
transactions: [[14803,22909,28,596,89,...,10831,2,2475,1919,4861]]

In [225]:
window_start_int64 = table.column("window_start")
window_start_timestamp = window_start_int64.cast(timestamp_type)

new_table = table.set_column(
        table.column_names.index('window_start'),
        'window_start',
        window_start_timestamp
)
new_table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
----
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,...,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z]]
transactions: [[14803,22909,28,596,89,...,10831,2,2475,1919,4861]]

In [226]:
new_table.schema

ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64

In [228]:
new_table.column("window_start").value_counts().tolist()

[{'values': Timestamp('2020-10-07 04:00:00+0000', tz='UTC'), 'counts': 8926}]

In [ ]:
character = 'A'
hex_code = hex(ord(character))
hex_string = hex_code[2:].upper()
print(hex_string)

In [346]:
def partition_key_escape(c: str) -> str:
    if c.islower():
        return "^" + c.upper()
    return "%" + "%02X" % ord(c)
def ticker_to_partition_key(ticker: str) -> str:
    if ticker.isupper():
        return ticker
    return "".join([f"{c if c.isupper() or c.isnumeric() else partition_key_escape(c)}" for c in ticker])
ticker_to_partition_key("BCp,L.9")

'BC^P%2CL%2E9'

In [347]:
# hive_partition_values = table.column('ticker')
# hive_partition_values.to_pylist()
hive_table = new_table.append_column('part', pa.array([ticker_to_partition_key(ticker) for ticker in table.column('ticker').to_pylist()]))
hive_table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
part: string
----
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,...,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z]]
transactions: [[14803,22909,28,596,89,...,10831,2,247

In [348]:
[s for s in hive_table.column('ticker').to_pylist() if s.startswith("BC")]

['BC',
 'BCpA',
 'BCpB',
 'BCpC',
 'BCAT',
 'BCBP',
 'BCC',
 'BCD',
 'BCDA',
 'BCDAW',
 'BCE',
 'BCEI',
 'BCEL',
 'BCH',
 'BCI',
 'BCLI',
 'BCM',
 'BCML',
 'BCO',
 'BCOR',
 'BCOV',
 'BCPC',
 'BCRX',
 'BCS',
 'BCSF',
 'BCTG',
 'BCV',
 'BCVpA',
 'BCX',
 'BCYC']

In [352]:
[s for s in hive_table.column('part').to_pylist() if s.startswith("AIG")]

['AIG', 'AIG^PA', 'AIG.WS']

In [351]:
[s for s in hive_table.column('ticker').to_pylist() if "." in s]

['ACND.U',
 'ACND.WS',
 'AGM.A',
 'AIG.WS',
 'AKO.A',
 'AKO.B',
 'AMBC.WS',
 'AONE.U',
 'AP.WS',
 'APSG.U',
 'ASAQ.U',
 'ASPL.U',
 'AVAN.U',
 'BF.A',
 'BF.B',
 'BFT.U',
 'BH.A',
 'BMRG.U',
 'BMRG.WS',
 'BRK.A',
 'BRK.B',
 'BRMK.WS',
 'BSN.U',
 'CCAC.WS',
 'CCIV.U',
 'CCIV.WS',
 'CCX.U',
 'CCX.WS',
 'CCXX.U',
 'CCXX.WS',
 'CIG.C',
 'CLA.U',
 'CLII.U',
 'CPSR.U',
 'CPSR.WS',
 'CRD.A',
 'CRD.B',
 'CRHC.U',
 'CWEN.A',
 'DEH.U',
 'DEH.WS',
 'DFNS.WS',
 'DGNR.U',
 'DGNR.WS',
 'DMS.WS',
 'DMYD.U',
 'DMYD.WS',
 'DMYT.U',
 'DMYT.WS',
 'EBR.B',
 'EMPW.U',
 'ENPC.U',
 'EQD.U',
 'FAII.U',
 'FAII.WS',
 'FEAC.U',
 'FEAC.WS',
 'FGNA.U',
 'FMAC.U',
 'FST.U',
 'FUSE.U',
 'FUSE.WS',
 'FVAC.U',
 'FVAC.WS',
 'GB.WS',
 'GEF.B',
 'GIK.WS',
 'GIX.WS',
 'GLEO.U',
 'GLEO.WS',
 'GOAC.U',
 'GOAC.WS',
 'GRP.U',
 'GSAH.U',
 'GSAH.WS',
 'GTN.A',
 'HEI.A',
 'HPX.U',
 'HPX.WS',
 'HVT.A',
 'HYLN.WS',
 'HZAC.U',
 'IACA.U',
 'IMPX.U',
 'IPOB.U',
 'IPOB.WS',
 'IPOC.U',
 'IPOC.WS',
 'IPV.U',
 'IPV.WS',
 'JIH.U',
 'JIH.WS'

In [232]:
config.by_ticker_dir

'/Volumes/Oahu/Mirror/files.polygon.io/flatfiles/us_stocks_sip/day_by_ticker_v1'

In [234]:
import os

In [236]:
ticker_count = len(new_table.column("ticker").unique())
ticker_count

8926

In [353]:
by_ticker_base_dir = os.path.join(
        config.by_ticker_dir,
        f"{config.agg_time}_{config.start_timestamp.date().isoformat()}_{config.end_timestamp.date().isoformat()}.hive",
    )
partition_by_ticker = pads.partitioning(
    pa.schema([("part", pa.string())]), flavor="hive"
)
pads.write_dataset(hive_table, base_dir=by_ticker_base_dir, format="parquet", partitioning=partition_by_ticker, max_partitions=ticker_count)

In [354]:
part_dataset = pads.dataset(by_ticker_base_dir, format="parquet", partitioning="hive")
part_table = part_dataset.to_table()
part_table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
part: string
----
ticker: [["A"],["AA"],...,["ZYNE"],["ZYXI"]]
volume: [[1322141],[3921229],...,[284131],[469532]]
open: [[102.47],[11.96],...,[3.51],[19.21]]
close: [[103.88],[12.24],...,[3.57],[19.38]]
high: [[104.75],[12.3],...,[3.6162],[19.73]]
low: [[102.42],[11.96],...,[3.4866],[19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z],[2020-10-07 04:00:00.000000000Z],...,[2020-10-07 04:00:00.000000000Z],[2020-10-07 04:00:00.000000000Z]]
transactions: [[14803],[22909],...,[1919],[4861]]
part: [["A"],["AA"],...,["ZYNE"],["ZYXI"]]

In [331]:
# part_table = part_table.sort_by("ticker")

In [355]:
hive_table = hive_table.sort_by("part")

In [357]:
# part_table.combine_chunks().equals(hive_table.combine_chunks())
part_table.sort_by("part").equals(hive_table)

True

In [243]:
one_chunk_table = part_table.combine_chunks()
one_chunk_table

pyarrow.Table
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
ticker: string
----
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,...,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z]]
transactions: [[14803,22909,28,596,89,...,10831,2,2475,1919,4861]]
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]

In [244]:
new_table.combine_chunks()

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
----
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,...,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z]]
transactions: [[14803,22909,28,596,89,...,10831,2,2475,1919,4861]]

In [242]:
part_table.combine_chunks().equals(new_table.combine_chunks())

False

In [245]:
# unchunked_table = new_table.unify_dictionaries()
unchunked_table = new_table.combine_chunks()
unchunked_table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
----
ticker: [["A","AA","AAA","AAAU","AACG",...,"ZVZZT","ZXIET","ZYME","ZYNE","ZYXI"]]
volume: [[1322141,3921229,3774,449029,6447,...,1100960,4000,225817,284131,469532]]
open: [[102.47,11.96,24.9017,18.8,1.115,...,10.05,100,43.99,3.51,19.21]]
close: [[103.88,12.24,24.9,18.8,1.1,...,10,100,45.06,3.57,19.38]]
high: [[104.75,12.3,24.91,18.82,1.115,...,10.05,100,45.68,3.6162,19.73]]
low: [[102.42,11.96,24.9,18.73,1.095,...,10,100,43.74,3.4866,19.02]]
window_start: [[2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,...,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z,2020-10-07 04:00:00.000000000Z]]
transactions: [[14803,22909,28,596,89,...,10831,2,2475,1919,4861]]

In [246]:
unchunked_table.column("window_start") == one_chunk_table.column("window_start")

False

MacOS apparently doesn't use ASCII/UTF8 collation order for sorting file names, or maybe it is something in Arrow datasets?
Also MacOS filenames are case insensitive by default so we're missing one of pairs like 'BCPC' and 'BCpC'.

In [341]:
# l1 = sorted(hive_table.column("ticker").to_pylist())
# l2 = sorted(part_table.column("ticker").to_pylist())
# l1 = hive_table.column("ticker").to_pylist()
# l2 = part_table.column("ticker").to_pylist()
l1 = hive_table.column("ticker").to_pylist()
l2 = part_table.column("ticker").to_pylist()
print(f"{len(l1)=} {len(l2)=}")
print(f"{len(l1)==len(l2)=}")
mismatches = [(i, l1[i], l2[i]) for i in range(min(len(l1), len(l2))) if l1[i] != l2[i]]
mismatches

len(l1)=8926 len(l2)=8926
len(l1)==len(l2)=True


[(34, 'ABR', 'ABRpA'),
 (35, 'ABRpA', 'ABRpB'),
 (36, 'ABRpB', 'ABRpC'),
 (37, 'ABRpC', 'ABR'),
 (74, 'ACND', 'ACND.U'),
 (75, 'ACND.U', 'ACND.WS'),
 (76, 'ACND.WS', 'ACND'),
 (132, 'AEL', 'AELpA'),
 (133, 'AELpA', 'AELpB'),
 (134, 'AELpB', 'AEL'),
 (187, 'AGM', 'AGMpC'),
 (188, 'AGMpC', 'AGMpD'),
 (189, 'AGMpD', 'AGMpF'),
 (190, 'AGMpF', 'AGM.A'),
 (191, 'AGM.A', 'AGM'),
 (198, 'AGO', 'AGOpB'),
 (199, 'AGOpB', 'AGOpE'),
 (200, 'AGOpE', 'AGOpF'),
 (201, 'AGOpF', 'AGO'),
 (216, 'AHH', 'AHHpA'),
 (217, 'AHHpA', 'AHH'),
 (222, 'AHT', 'AHTpD'),
 (223, 'AHTpD', 'AHTpF'),
 (224, 'AHTpF', 'AHTpG'),
 (225, 'AHTpG', 'AHTpH'),
 (226, 'AHTpH', 'AHTpI'),
 (227, 'AHTpI', 'AHT'),
 (228, 'AI', 'AIpB'),
 (229, 'AIpB', 'AIpC'),
 (230, 'AIpC', 'AI'),
 (235, 'AIG', 'AIGpA'),
 (236, 'AIGpA', 'AIG.WS'),
 (237, 'AIG.WS', 'AIG'),
 (277, 'AL', 'ALpA'),
 (278, 'ALpA', 'AL'),
 (300, 'ALL', 'ALLpB'),
 (301, 'ALLpB', 'ALLpG'),
 (302, 'ALLpG', 'ALLpH'),
 (303, 'ALLpH', 'ALLpI'),
 (304, 'ALLpI', 'ALL'),
 (309, 'ALL

In [ ]:
mismatches

In [ ]:
filtered_table = new_table.filter(pa.compute.(table['new_table'], 'a'))

In [264]:
missing_tickers = set(l1) - set(l2)
missing_tickers

{'BCPC', 'CPK', 'CPS', 'TPC'}

In [268]:
df = new_table.to_pandas()
df[df['ticker'].isin(missing_tickers)]

,ticker,volume,open,close,high,low,window_start,transactions
809,BCPC,132874,101.85,101.14,103.50,101.14,2020-10-07 04:00:00+00:00,2820
1728,CPK,128262,86.67,86.31,86.80,85.24,2020-10-07 04:00:00+00:00,2309
1734,CPS,118020,15.62,16.12,16.36,15.62,2020-10-07 04:00:00+00:00,2633
7940,TPC,537608,12.86,13.61,13.88,12.82,2020-10-07 04:00:00+00:00,3859


In [283]:
'BCpC' in df['ticker'].unique(), 'BCPC' in df['ticker'].unique()

(True, True)

In [272]:
df[df['ticker'].apply(lambda t: t.lower in set(x.lower for x in missing_tickers))]

,ticker,volume,open,close,high,low,window_start,transactions


In [270]:
df2 = one_chunk_table.to_pandas()
df2[df2['ticker'].isin(missing_tickers)]

,volume,open,close,high,low,window_start,transactions,ticker


In [282]:
'BCpC' in df2['ticker'].unique(), 'BCPC' in df2['ticker'].unique()

(True, False)

In [252]:
unchunked_table.column("ticker").to_pylist() == one_chunk_table.column("ticker").to_pylist()

False

In [160]:
sorted_table = unchunked_table.sort_by([("ticker", "ascending"), ("window_start", "ascending")])
unchunked_table.equals(sorted_table)

False

In [206]:
sorted_indicies = pyarrow.compute.sort_indices(new_table, sort_keys=[("ticker", "ascending"), ("window_start", "ascending")])
sorted_indicies

[
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  ...
  1420793,
  1420794,
  1420795,
  1420796,
  1420797,
  1420798,
  1420799,
  1420800,
  1420801,
  1420802
]

In [207]:
deltas = pyarrow.compute.pairwise_diff(sorted_indicies)
deltas

[
  null,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  ...
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1
]

In [229]:
deltas.drop_null().value_counts().tolist()

[{'values': 1, 'counts': 1420550},
 {'values': 30, 'counts': 4},
 {'values': 18446744073709551571, 'counts': 1},
 {'values': 18, 'counts': 2},
 {'values': 3, 'counts': 1},
 {'values': 47, 'counts': 1},
 {'values': 18446744073709551474, 'counts': 1},
 {'values': 98, 'counts': 1},
 {'values': 18446744073709547632, 'counts': 1},
 {'values': 3984, 'counts': 1},
 {'values': 46, 'counts': 4},
 {'values': 277, 'counts': 1},
 {'values': 18446744073709549836, 'counts': 1},
 {'values': 1506, 'counts': 1},
 {'values': 29, 'counts': 1},
 {'values': 18446744073709551537, 'counts': 1},
 {'values': 53, 'counts': 3},
 {'values': 18446744073709542097, 'counts': 1},
 {'values': 9476, 'counts': 1},
 {'values': 90, 'counts': 1},
 {'values': 18446744073709551525, 'counts': 1},
 {'values': 4, 'counts': 8},
 {'values': 60, 'counts': 1},
 {'values': 18446744073709551280, 'counts': 1},
 {'values': 279, 'counts': 1},
 {'values': 181, 'counts': 1},
 {'values': 18446744073709550846, 'counts': 1},
 {'values': 592,

In [161]:
sorted_table

pyarrow.Table
ticker: string
volume: int64
open: double
close: double
high: double
low: double
window_start: timestamp[ns, tz=UTC]
transactions: int64
----
ticker: [["A","A","A","A","A",...,"ZYXI","ZYXI","ZYXI","ZYXI","ZYXI"]]
volume: [[714,9178,177,529,3206,...,4158,2192,4779,8271,6232]]
open: [[101.6537,102.47,102.49,102.925,102.88,...,19.44,19.445,19.45,19.4196,19.38]]
close: [[101.6537,102.44,102.49,102.98,103,...,19.445,19.45,19.42,19.41,19.38]]
high: [[101.6537,102.47,102.49,102.98,103,...,19.445,19.45,19.45,19.42,19.38]]
low: [[101.6537,102.42,102.49,102.92,102.84,...,19.43,19.445,19.42,19.38,19.38]]
window_start: [[2020-10-07 13:09:00.000000000Z,2020-10-07 13:30:00.000000000Z,2020-10-07 13:31:00.000000000Z,2020-10-07 13:32:00.000000000Z,2020-10-07 13:33:00.000000000Z,...,2020-10-07 19:56:00.000000000Z,2020-10-07 19:57:00.000000000Z,2020-10-07 19:58:00.000000000Z,2020-10-07 19:59:00.000000000Z,2020-10-07 20:00:00.000000000Z]]
transactions: [[1,16,8,12,67,...,55,48,114,104,12]]

In [149]:
sorted_table.shape

(1420803, 8)

In [193]:
unchunked_table.column("window_start")

[
  [
    2020-10-07 13:09:00.000000000Z,
    2020-10-07 13:30:00.000000000Z,
    2020-10-07 13:31:00.000000000Z,
    2020-10-07 13:32:00.000000000Z,
    2020-10-07 13:33:00.000000000Z,
    ...
    2020-10-07 19:56:00.000000000Z,
    2020-10-07 19:57:00.000000000Z,
    2020-10-07 19:58:00.000000000Z,
    2020-10-07 19:59:00.000000000Z,
    2020-10-07 20:00:00.000000000Z
  ]
]

In [195]:
unchunked_table.column("window_start") == sorted_table.column("window_start")

True

In [200]:
unchunked_table.column("window_start").to_array().diff(sorted_table.column("window_start").to_array())

AttributeError: 'pyarrow.lib.ChunkedArray' object has no attribute 'to_array'

In [203]:
import pyarrow

In [205]:
pyarrow.compute.pairwise_diff(unchunked_table.column("window_start").to_array())

AttributeError: 'pyarrow.lib.ChunkedArray' object has no attribute 'to_array'

In [198]:
unchunked_table.column("ticker").sort() == sorted_table.column("ticker")

True

In [162]:
df = unchunked_table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420803 entries, 0 to 1420802
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   ticker        1420803 non-null  object             
 1   volume        1420803 non-null  int64              
 2   open          1420803 non-null  float64            
 3   close         1420803 non-null  float64            
 4   high          1420803 non-null  float64            
 5   low           1420803 non-null  float64            
 6   window_start  1420803 non-null  datetime64[ns, UTC]
 7   transactions  1420803 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2), object(1)
memory usage: 86.7+ MB


,ticker,volume,open,close,high,low,window_start,transactions
0,A,714,101.6537,101.6537,101.6537,101.6537,2020-10-07 13:09:00+00:00,1
1,A,9178,102.4700,102.4400,102.4700,102.4200,2020-10-07 13:30:00+00:00,16
2,A,177,102.4900,102.4900,102.4900,102.4900,2020-10-07 13:31:00+00:00,8
3,A,529,102.9250,102.9800,102.9800,102.9200,2020-10-07 13:32:00+00:00,12
4,A,3206,102.8800,103.0000,103.0000,102.8400,2020-10-07 13:33:00+00:00,67
...,...,...,...,...,...,...,...,...
1420798,ZYXI,4158,19.4400,19.4450,19.4450,19.4300,2020-10-07 19:56:00+00:00,55
1420799,ZYXI,2192,19.4450,19.4500,19.4500,19.4450,2020-10-07 19:57:00+00:00,48
1420800,ZYXI,4779,19.4500,19.4200,19.4500,19.4200,2020-10-07 19:58:00+00:00,114
1420801,ZYXI,8271,19.4196,19.4100,19.4200,19.3800,2020-10-07 19:59:00+00:00,104


In [170]:
df.reset_index(inplace=True)
df.index

RangeIndex(start=0, stop=1420803, step=1)

In [179]:
df1 = df.sort_values(by=["ticker", "window_start"])

In [174]:
df.index = list(df.index)

In [175]:
df.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1420793, 1420794, 1420795, 1420796, 1420797, 1420798, 1420799, 1420800,
       1420801, 1420802],
      dtype='int64', length=1420803)

In [180]:
diff_df = df.compare(df1)
print(f"{len(diff_df)=}")
diff_df

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [181]:
df.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1420793, 1420794, 1420795, 1420796, 1420797, 1420798, 1420799, 1420800,
       1420801, 1420802],
      dtype='int64', length=1420803)

In [182]:
df1.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1420793, 1420794, 1420795, 1420796, 1420797, 1420798, 1420799, 1420800,
       1420801, 1420802],
      dtype='int64', length=1420803)

In [184]:
df.index.compare(df1.index)

AttributeError: 'Index' object has no attribute 'compare'

In [137]:
df['window_start']

0         2020-10-07 13:09:00+00:00
1         2020-10-07 13:30:00+00:00
2         2020-10-07 13:31:00+00:00
3         2020-10-07 13:32:00+00:00
4         2020-10-07 13:33:00+00:00
                     ...           
1420798   2020-10-07 19:56:00+00:00
1420799   2020-10-07 19:57:00+00:00
1420800   2020-10-07 19:58:00+00:00
1420801   2020-10-07 19:59:00+00:00
1420802   2020-10-07 20:00:00+00:00
Name: window_start, Length: 1420803, dtype: datetime64[ns, UTC]

In [138]:
df1['window_start']

0         2020-10-07 13:09:00+00:00
1         2020-10-07 13:30:00+00:00
2         2020-10-07 13:31:00+00:00
3         2020-10-07 13:32:00+00:00
4         2020-10-07 13:33:00+00:00
                     ...           
1420798   2020-10-07 19:56:00+00:00
1420799   2020-10-07 19:57:00+00:00
1420800   2020-10-07 19:58:00+00:00
1420801   2020-10-07 19:59:00+00:00
1420802   2020-10-07 20:00:00+00:00
Name: window_start, Length: 1420803, dtype: datetime64[ns, UTC]

In [183]:
df['window_start'].compare(df1['window_start'])

ValueError: Can only compare identically-labeled Series objects

In [186]:
df['ticker']

0             A
1             A
2             A
3             A
4             A
           ... 
1420798    ZYXI
1420799    ZYXI
1420800    ZYXI
1420801    ZYXI
1420802    ZYXI
Name: ticker, Length: 1420803, dtype: object

In [187]:
df1['ticker']

0             A
1             A
2             A
3             A
4             A
           ... 
1420798    ZYXI
1420799    ZYXI
1420800    ZYXI
1420801    ZYXI
1420802    ZYXI
Name: ticker, Length: 1420803, dtype: object

In [188]:
df1['ticker'].compare(df1['ticker'])

,self,other


In [190]:
df['ticker'].equals(df['ticker'])

True

In [173]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420803 entries, 0 to 1420802
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   index         1420803 non-null  int64              
 1   ticker        1420803 non-null  object             
 2   volume        1420803 non-null  int64              
 3   open          1420803 non-null  float64            
 4   close         1420803 non-null  float64            
 5   high          1420803 non-null  float64            
 6   low           1420803 non-null  float64            
 7   window_start  1420803 non-null  datetime64[ns, UTC]
 8   transactions  1420803 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(3), object(1)
memory usage: 97.6+ MB


,index,ticker,volume,open,close,high,low,window_start,transactions
0,0,A,714,101.6537,101.6537,101.6537,101.6537,2020-10-07 13:09:00+00:00,1
1,1,A,9178,102.4700,102.4400,102.4700,102.4200,2020-10-07 13:30:00+00:00,16
2,2,A,177,102.4900,102.4900,102.4900,102.4900,2020-10-07 13:31:00+00:00,8
3,3,A,529,102.9250,102.9800,102.9800,102.9200,2020-10-07 13:32:00+00:00,12
4,4,A,3206,102.8800,103.0000,103.0000,102.8400,2020-10-07 13:33:00+00:00,67
...,...,...,...,...,...,...,...,...,...
1420798,1420798,ZYXI,4158,19.4400,19.4450,19.4450,19.4300,2020-10-07 19:56:00+00:00,55
1420799,1420799,ZYXI,2192,19.4450,19.4500,19.4500,19.4450,2020-10-07 19:57:00+00:00,48
1420800,1420800,ZYXI,4779,19.4500,19.4200,19.4500,19.4200,2020-10-07 19:58:00+00:00,114
1420801,1420801,ZYXI,8271,19.4196,19.4100,19.4200,19.3800,2020-10-07 19:59:00+00:00,104


In [141]:
df1.info()
df1

<class 'pandas.core.frame.DataFrame'>
Index: 1420803 entries, 0 to 1420802
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   ticker        1420803 non-null  object             
 1   volume        1420803 non-null  int64              
 2   open          1420803 non-null  float64            
 3   close         1420803 non-null  float64            
 4   high          1420803 non-null  float64            
 5   low           1420803 non-null  float64            
 6   window_start  1420803 non-null  datetime64[ns, UTC]
 7   transactions  1420803 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2), object(1)
memory usage: 97.6+ MB


,ticker,volume,open,close,high,low,window_start,transactions
0,A,714,101.6537,101.6537,101.6537,101.6537,2020-10-07 13:09:00+00:00,1
1,A,9178,102.4700,102.4400,102.4700,102.4200,2020-10-07 13:30:00+00:00,16
2,A,177,102.4900,102.4900,102.4900,102.4900,2020-10-07 13:31:00+00:00,8
3,A,529,102.9250,102.9800,102.9800,102.9200,2020-10-07 13:32:00+00:00,12
4,A,3206,102.8800,103.0000,103.0000,102.8400,2020-10-07 13:33:00+00:00,67
...,...,...,...,...,...,...,...,...
1420798,ZYXI,4158,19.4400,19.4450,19.4450,19.4300,2020-10-07 19:56:00+00:00,55
1420799,ZYXI,2192,19.4450,19.4500,19.4500,19.4450,2020-10-07 19:57:00+00:00,48
1420800,ZYXI,4779,19.4500,19.4200,19.4500,19.4200,2020-10-07 19:58:00+00:00,114
1420801,ZYXI,8271,19.4196,19.4100,19.4200,19.3800,2020-10-07 19:59:00+00:00,104
